### Earthquake Location

In [1]:
import sys
import os
import numpy as np
import pandas as pd
from scipy import signal
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
#Station data
lat=np.array([35.0,-44.0,-11.0,23.0,42.0,-12.0,-45.0,5.0,-1.0,20.0])
lon=np.array([9.0,10.0,-25.0,-39.0,-27.0,50.0,16.0,-19.0,-11.0,11.0])
depth=np.zeros(10)

n=len(lat) ##number of observations
m=4        ##number of parameters

#Observed data (from station and  pridicted by model)
#Initialization
Station_data=np.zeros(10)
d=np.zeros(n)
delta_d=np.zeros(n)

#Actual Value of model parameters to compute observed data at stations
lat_0=0.0
lon_0=0.0
depth_0=10.0
time=0.0
vel=5.0

#Starting model=[3,4,20,2] : model parameters=(x,y,z,t)
x=3.0
y=4.0
z=20.0
t=2.0

In [3]:
#Arrival_time Equation
def compute(lat_0,lon_0,depth_0,time,lat,lon,depth):
    return np.sqrt((1/vel *((lat_0-lat)**2 + (lon_0-lon)**2 + (depth_0-depth)**2))) + time
                   
for i in range(n):                   
    Station_data[i]=compute(lat_0,lon_0,depth_0,time,lat[i],lon[i],depth[i])  #Generate Data from the ten stations
#Station_data

In [4]:
## Compute Design matrix
G=np.zeros((n,m))
G_x=np.zeros(n)
G_y=np.zeros(n)
G_z=np.zeros(n)
G_t=np.zeros(n)

In [5]:
for k in range(3):             # 3 iterations
    for i in range(n):  
        d[i]=compute(x,y,z,t,lat[i],lon[i],depth[i])                              #Generate Data from starting model
        delta_d[i]= Station_data[i] - d[i]                                        # Compute misfit(difference between original data and predicted data)
    
        #Partial-Derivative matrix about the starting model
        G_x[i]=(x-lat[i])/vel * ((x-lat[i])**2.0 + (y-lon[i])**2.0 + z**2.0)**-0.5

        G_y[i]=(y-lon[i])/vel * ((x-lat[i])**2.0 + (y-lon[i])**2.0 + z**2.0)**-0.5

        G_z[i]=(z)/vel * ((x-lat[i])**2.0 + (y-lon[i])**2.0 + z**2.0)**-0.5

        G_t[i]=1
    
        #A=design matrix
        G[i][0]=G_x[i]
        G[i][1]=G_y[i]
        G[i][2]=G_z[i]
        G[i][3]=G_t[i]   
            
   #Least Square Estimation
    G_tran=np.transpose(G)
    N=np.matmul(G_tran,G)
    N_inv=np.linalg.inv(N)
    G1=np.matmul(N_inv,G_tran)

    #Estimated parameters
    X_parameters=np.matmul(G1,delta_d)

    delta_x=X_parameters[0]  #a=
    delta_y=X_parameters[1]  #b=slope
    delta_z=X_parameters[2]  #
    delta_t=X_parameters[3]  # 
    
    #modified model parameters
    x=x+delta_x
    y=y+delta_y
    z=z+delta_z
    t=t+delta_t
    
    
# print(delta_x,delta_y,delta_z,delta_t)   
    print(x,y,z,t)

-4.919883868313254 -6.314098525744162 -2.04018377056277 0.5034004679247497
5.68812600414494 7.616068453114279 -45.92015517870317 0.29599004017752395
-16.602732330445203 -20.76870304860867 61.832828604724654 7.731272300987723
